In [ ]:
SIE510-GIS Applications
Lab8: Market Area Analysis
- Eugenie V. Kamgue -

In [1]:
import arcpy
from arcpy import env

env.workspace=r"c:\Lab8\Lab8.gdb"
env.overwriteOutput=True


In [90]:
infile=r"c:\Lab8\Lab8.gdb\newArea"

In [100]:
fields=arcpy.ListFields(infile)

field_names=[f.name for f in arcpy.ListFields(infile)]
print (field_names)

['OBJECTID', 'Shape', 'FID_bbvblocks10', 'GEOID10', 'STATEFP10', 'COUNTY', 'COUNTYFP10', 'COUSUB10', 'COUSUB10NA', 'TRACT10', 'BLOCK10', 'BLKGRP10', 'BLKNAME', 'ALAND10', 'AWATER10', 'POP10', 'USDSTRCT03', 'HSDSTRCT03', 'SNDSTRCT03', 'Blk_Area', 'FID_SAPolygons3', 'FacilityID', 'Name', 'FromBreak', 'ToBreak', 'Shape_Length', 'Shape_Area', 'Ratio', 'Calc_Pop', 'testArea']


In [10]:
import pandas as pd
from arcgis import GIS

In [104]:
blks=pd.DataFrame.spatial.from_featureclass(r'c:\Lab8\Lab8.gdb\newArea')
blks.head()

,OBJECTID,FID_bbvblocks10,GEOID10,STATEFP10,COUNTY,COUNTYFP10,COUSUB10,COUSUB10NA,TRACT10,BLOCK10,...,Blk_Area,FID_SAPolygons3,FacilityID,Name,FromBreak,ToBreak,Ratio,Calc_Pop,testArea,SHAPE
0,1,-1,,,,,,,,,...,0.000000,1,4,Location 4 : 0 - 5,0.0,5.0,NaN,NaN,392136.968750,"{""rings"": [[[516054.44859999977, 4958434.2665]..."
1,2,1,230190003003008,23,Penobscot,019,02795,Bangor city,000300,3008,...,31159.539399,3,1,Location 1 : 0 - 5,0.0,5.0,1.0,129.0,31159.539062,"{""rings"": [[[518672.58949999977, 4961771.08070..."
2,3,2,230190004001057,23,Penobscot,019,02795,Bangor city,000400,1057,...,9437.059162,3,1,Location 1 : 0 - 5,0.0,5.0,1.0,42.0,9437.059570,"{""rings"": [[[518099.6191999996, 4961453.8511],..."
3,4,3,230190002002006,23,Penobscot,019,02795,Bangor city,000200,2006,...,753.098526,3,1,Location 1 : 0 - 5,0.0,5.0,1.0,0.0,753.098511,"{""rings"": [[[518245.3958999999, 4961025.144099..."
4,5,4,230190005002013,23,Penobscot,019,02795,Bangor city,000500,2013,...,10247.382497,3,1,Location 1 : 0 - 5,0.0,5.0,1.0,24.0,10247.382812,"{""rings"": [[[517521.6140999999, 4961367.3815],..."


In [105]:
blks.shape

(1046, 28)

In [96]:
arcpy.AddField_management(infile, "testArea", "Float", field_precision=12, field_scale=4) # adding a field to store the computed service area values

<Result 'c:\\Lab8\\Lab8.gdb\\newArea'>

In [36]:
def polygonArea(x,y): # x is a list of x coords, y is a list of y coords by polygon. Using the function polygonArea to calculate surfaces of polygons comprised in the service area
    area=0
    count=len(x)
    j=count-1
    for i in range (0, count):
      area+=((x[j]+x[i])*(y[j]-y[i]))
      j=i
    return area*.5

In [97]:
xlist=[]
ylist=[]
with arcpy.da.UpdateCursor(infile, ["SHAPE@", "testArea"]) as ucur:
    for row in ucur:
        # Step through each part of the feature
        for part in row[0]:
            for pnt in part:
                if pnt:
                    #
                    xlist.append(pnt.X) 
                    ylist.append(pnt.Y)
                else:
                    # If pnt is None, this represents an interior ring
                    print("Interior Ring:")
        pa=polygonArea(xlist, ylist)
        print (pa)
        row[1]= pa
        ucur.updateRow(row)
        xlist.clear()
        ylist.clear()

392136.9635477066
31159.539399050176
9437.059161958285
753.0985257430002
10247.382497167215
12844.123923063744
4568.938000159804
9985.596893273294
2584.7090981705114
11529.283423525281
9367.48330804333
13668.855948649347
77305.64561045995
16365.602242423221
18121.0610608547
22907.894069775008
19553.532962469384
6657.935441390611
10678.47614261508
4286.946817727527
23923.2450341098
44926.03317954112
26890.401862803847
1913.8102438165806
235.5934196318267
15747.760392258875
5708.469004690647
298.6058024032973
453.784546000883
12134.684170448221
11994.339994194917
194031.9085103029
228347.48688700423
1465.1661831103265
2181605.973863706
30156.9728297377
186124.90842799097
6364.730278097093
14851.255227774382
15882.64776417613
1278.9621037896723
11021.139598833397
8651.085922706872
48333.62300352007
31195.927977939136
13524.71551825665
10504.333382545039
16199.791342820972
2313.9194694133475
32500.10041426681
590.4417367521673
39424.55660805944
12154.374987525865
25257.713140547276
704496.

20085.092523628846
Interior Ring:
252490.66288971342
2717.328809250146
21189.074981162325
15188.814338951372
17617.838630067185
16906.754935257137
416622.93896974344
287677.41496982425
Interior Ring:
285094.5892410353
40800.33369578794
13999.927436180878
287121.1119675115
24777.399633470923
16630.751103548333
12319.113218069077
10994.95664501004
28222.521046916023
1451.244450069964
16341.676102891564
27187.80120249279
1827.5183633118868
12035.753353727516
10305.374118756037
9087.348795809783
22708.685085082427
1692.9862552825361
29010.558427730575
15981.451487481012
30703.20086750388
36143.27247188147
30800.716628497466
29905.792677877005
154519.48918894585
22403.161506980658
45164.18048205413
405.2378655169159
2364.4297744603828
5554.833143193275
304622.9998028986
219549.74079516437
2904.689597249031
10810.791803656844
Interior Ring:
Interior Ring:
Interior Ring:
799180.5095337913
15434.223989523947
2088.003724297043
39013.861107483506
9354.984073443338
2.007101430092007
12802.7676375

-47573.77503010444
1417.1775451251306
1417.1775451251306
-18737.176931120455
-18737.176931120455
272.38223441690207
272.38223441690207
56.15484806057066
56.15484806057066
3609.3272692803293
3609.3272692803293
789.2243524985388
789.2243524985388
7054.535923426083
7054.535923426083
301.0530174198793
301.0530174198793
578.307006508112
578.307006508112
-216242.2443954721
-216242.2443954721
956.9080906263553
956.9080906263553
1538.8525184467435
1538.8525184467435
165.5245683118701
165.5245683118701
67.92554853460751
67.92554853460751
274.55792149639456
274.55792149639456
1956.8803005600348
1956.8803005600348
1660.682166219689
1660.682166219689
12562.638641493395
12562.638641493395
707.2312926352024
707.2312926352024
1631.1922298464924
1631.1922298464924
1.2792869742870607
1.2792869742870607
1662.9443115405738
1662.9443115405738
5241.552857601084
5241.552857601084
302.23433529026806
302.23433529026806
329.4902047002688
329.4902047002688
394.436032471247
394.436032471247
339.6383411887218
339

In [92]:
arcpy.AddField_management(infile, "Ratio", "Float", field_precision=8, field_scale=4) # adding a field to hold the ratio between the populated area and the shape area

<Result 'c:\\Lab8\\Lab8.gdb\\newArea'>

In [93]:
arcpy.CalculateField_management(infile, "Ratio", 
                                '!Shape_Area!/!Blk_Area!', "PYTHON3")  # to compute the ratio between the populated area and the shape area

ZeroDivisionError: float division by zero

<Result 'c:\\Lab8\\Lab8.gdb\\newArea'>

In [94]:
arcpy.AddField_management(infile, "Calc_Pop", "Float", field_precision=8, field_scale=4)  #adding a field to hold the population within the service area

<Result 'c:\\Lab8\\Lab8.gdb\\newArea'>

In [95]:
arcpy.CalculateField_management(infile, "Calc_Pop", 
                                '!Ratio!*!POP10!', "PYTHON3")     #to find the population within the service area

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

<Result 'c:\\Lab8\\Lab8.gdb\\newArea'>

In [99]:
out_table="summaryTable"  # Creating a table that summarizes the calculated service area and the adjusted population within that area of interest
stat_fields=[['Calc_Pop', 'SUM'], ['testArea', 'SUM']] 
stats=arcpy.Statistics_analysis(infile,out_table, stat_fields, case_field='FacilityID')

In [101]:
field_names=[i.name for i in arcpy.ListFields(out_table) if i.type !='OID']
cursor=arcpy.da.SearchCursor(out_table, field_names)
stats=pd.DataFrame(data=[row for row in cursor], columns=field_names)
stats.head()

,FacilityID,FREQUENCY,SUM_Calc_Pop,SUM_testArea
0,1,458,15707.479746,1.468094e+07
1,2,119,1704.986283,5.085632e+06
2,3,98,5016.033869,1.108535e+07
3,4,371,10248.785495,1.697079e+07


In [102]:
stats.shape

(4, 4)

In [106]:
field=["SUM_testArea"]
with arcpy.da.UpdateCursor(out_table, field) as cursor:
    for row in cursor:
        row[0]=row[0]/1000000  # To convert the testArea in square kilometers
        cursor.updateRow(row)

In [107]:
field_names=[i.name for i in arcpy.ListFields(out_table) if i.type !='OID']
cursor=arcpy.da.SearchCursor(out_table, field_names)
stats=pd.DataFrame(data=[row for row in cursor], columns=field_names)
stats.head()

,FacilityID,FREQUENCY,SUM_Calc_Pop,SUM_testArea
0,1,458,15707.479746,14.680936
1,2,119,1704.986283,5.085632
2,3,98,5016.033869,11.085349
3,4,371,10248.785495,16.970791


In [108]:
arcpy.AlterField_management(out_table, "SUM_Calc_Pop", "Population", "Population")  # renaming field

<Result 'c:\\Lab8\\Lab8.gdb\\summaryTable'>

In [109]:
arcpy.AlterField_management(out_table, "SUM_testArea", "Service_Area", "Service_Area")  #renaming field

<Result 'c:\\Lab8\\Lab8.gdb\\summaryTable'>

In [110]:
field_names=[i.name for i in arcpy.ListFields(out_table) if i.type !='OID']
cursor=arcpy.da.SearchCursor(out_table, field_names)
stats=pd.DataFrame(data=[row for row in cursor], columns=field_names)
stats.head()

,FacilityID,FREQUENCY,Population,Service_Area
0,1,458,15707.479746,14.680936
1,2,119,1704.986283,5.085632
2,3,98,5016.033869,11.085349
3,4,371,10248.785495,16.970791


In [111]:
arcpy.DeleteField_management(out_table, "FREQUENCY")   # deleting the field FREQUENCY because not needed

<Result 'c:\\Lab8\\Lab8.gdb\\summaryTable'>

In [112]:
field_names=[i.name for i in arcpy.ListFields(out_table) if i.type !='OID']
cursor=arcpy.da.SearchCursor(out_table, field_names)
stats=pd.DataFrame(data=[row for row in cursor], columns=field_names)
stats.head()

,FacilityID,Population,Service_Area
0,1,15707.479746,14.680936
1,2,1704.986283,5.085632
2,3,5016.033869,11.085349
3,4,10248.785495,16.970791
